# Adversarial Search: Playing Connect 4


## Instructions

Total Points: Undegraduates 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play Connect 4:

> "Connect 4 is a two-player connection board game, in which the players choose a color and then take turns dropping colored discs into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs." (see [Connect Four on Wikipedia](https://en.wikipedia.org/wiki/Connect_Four))

## Task 1: Defining the Search Problem [1 point]

Define the components of the search problem:

* Initial state - Empty board of six rows and seven columns
* Actions - Selection of column on the board
* Transition model - Places agent's piece in the lowest available slot of selected column
* Goal state - Horizontal, vertical, or diagonal line of agent's pieces

How big is the search space?

The search space contains between C(42,21) and 3^42 states. The lower bound is the number of ways to fill a 6 by 7 board with the 21 pieces available to each agent. There are more possible states, as before the board reaches completely filled states, some locations on the board will be blank. The upper bound is the number of ways to fill a 6 by 7 grid with blanks or pieces from either player. Some of the states included in the upper bound are not possible, as blanks can only occupy the top of each column.

## Task 2: Game Environment and Random Agent [2 point]

Use a numpy character array as the board.

In [1]:
import numpy as np

def empty_board(shape=(6, 7)):
    return np.full(shape=shape, fill_value=' ')

print(empty_board())

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']]


Instead of colors for the players use 'x' and 'o' to represent the players. Make sure that your agent functions all have the form: `agent_type(board, player = 'x')`, where board is the current board position and player is the player whose next move it is and who the agent should play.

Implement the board and helper functions for:

* The transition model (result).
* The utility function.
* Check for terminal states.
* A check for available actions.
* A function to visualize the board.

Make sure that all these functions work with boards of different sizes.

Implement an agent that plays randomly and let two random agents play against each other 1000 times. How often does each player win? Is the result expected? 

In [2]:
from functools import lru_cache, wraps

def np_cache(*args, **kwargs):
    ''' LRU cache implementation for functions whose FIRST parameter is a numpy array
        forked from: https://gist.github.com/Susensio/61f4fee01150caaac1e10fc5f005eb75 '''

    def decorator(function):
        @wraps(function)
        def wrapper(np_array, *args, **kwargs):
            hashable_array = tuple(map(tuple, np_array))
            return cached_wrapper(hashable_array, *args, **kwargs)

        @lru_cache(*args, **kwargs)
        def cached_wrapper(hashable_array, *args, **kwargs):
            array = np.array(hashable_array)
            return function(array, *args, **kwargs)

        # copy lru_cache attributes over too
        wrapper.cache_info = cached_wrapper.cache_info
        wrapper.cache_clear = cached_wrapper.cache_clear
        return wrapper
    return decorator

In [3]:
# Helper functions

from re import search

def drop(board, piece, column, inplace=True):
    board_ = np.array(board, copy=not inplace)
    for row in reversed(board_):
        if row[column] == ' ':
            row[column] = piece
            return board_

def utility(board, player, length=4):
    """Determines utility of board for player if terminal"""
    state = terminal(board, player, length)
    if state == player: return 1
    if state == 'draw': return 0
    if state:           return -1

def terminal(board, last_move=None, length=4):
    """Returns piece of the player that has won.
    Returns 'draw' if draw, otherwise board is not
    in terminal state and returns None."""

    cols = board.shape[1]

    chars = [last_move]
    if last_move is None:
        chars = np.unique(board).tolist()
        if ' ' in chars: chars.remove(' ')

    # unravel board into string
    board_str = '|'.join(str(row.tostring().replace(b'\x00', b'').decode("utf-8")) for row in board)
    
    # check for four in a line
    for char in chars:
        if search(char * length, board_str):
            # horizontal
            return char
        if search(char + ('.' * cols + char) * (length - 1), board_str):
            # vertical
            return char
        if search(char + ('.' * (cols + 1) + char) * (length - 1), board_str):
            # diagonal top-left to bottom-right
            return char
        if search(char + ('.' * (cols - 1) + char) * (length - 1), board_str):
            # diagonal top-right to bottom-left
            return char
    
    # if no matches and all columns are filled,
    # then draw has been reached
    if ' ' not in board:
        return 'draw'

    # if none of previous conditions are satisfied,
    # board is not in terminal state.
    return None

def actions(board):
    actions = []
    for column, space in enumerate(board[0]):
        if space == ' ':
            actions.append(column)
    return actions

@lru_cache
def other(player):
    return 'o' if player == 'x' else 'x'

In [4]:
# Random Agent implementation

import random

def random_agent(board, player='x'):
    move = random.choice(actions(board))
    drop(board, player, move)

In [5]:
# Play two random agents against each other 1000 times

def play(agent1, agent2, size=(6,7), n=1000):
    agents = {'x': agent1, 'o': agent2}
    games = {'x': 0, 'o': 0, 'draw': 0}
    for i in range(n):
        board = empty_board(size)
        player = None
        state = 0
        while not state:
            player = other(player)
            agents[player](board, player)
            state = terminal(board, player)
            print(board)
        games[state] += 1
    return games

' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ' ' ' ' ']
 ['o' ' ' 'x' ' ' ' ' ' ' 'x']
 ['o' 'x' 'o' ' ' ' ' ' ' 'o']
 ['x' 'o' 'x' 'o' ' ' 'o' 'x']]
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ' ' ' ' ']
 ['o' ' ' 'x' ' ' ' ' ' ' 'x']
 ['o' 'x' 'o' 'x' ' ' ' ' 'o']
 ['x' 'o' 'x' 'o' ' ' 'o' 'x']]
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ' ' ' ' ']
 ['o' ' ' 'x' ' ' ' ' ' ' 'x']
 ['o' 'x' 'o' 'x' ' ' ' ' 'o']
 ['x' 'o' 'x' 'o' 'o' 'o' 'x']]
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ' ' ' ' ']
 ['o' ' ' 'x' ' ' ' ' ' ' 'x']
 ['o' 'x' 'o' 'x' ' ' ' ' 'o']
 ['x' 'o' 'x' 'o' 'o' 'o' 'x']]
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ' ' ' ' ']
 ['o' ' ' 'x' ' ' ' ' ' ' 'x']
 ['o' 'x' 'o' 'x' ' ' 'o' 'o']
 ['x' 'o' 'x' 'o' 'o' 'o' 'x']]
[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ' ' ' ' ']
 [' ' ' ' '

KeyboardInterrupt: 

Player `'x'` appears to win 55% of the time, and player `'o'` appears to win 45% of the time. Having the first turn appears to give an advantage.

## Task 3: Minimax Search with Alpha-Beta Pruning [4 points]

### Implement the search starting from a given board and specifying the player.



__Note:__ The search space for a $6 \times 7$ board is large. You can experiment with smaller boards (the smallest is $4 \times 4$) and/or changing the winning rule to connect 3 instead of 4.

In [8]:
import math

def minimax_agent(board, player='x'):
    move, _ = best_move(board, player)
    drop(board, player, move)

@np_cache(maxsize=2**22)
def best_move(board, player, alpha=-math.inf, beta=+math.inf):
    """Returns best move of player and its value"""
    value = utility(board, player)
    if value is not None: return None, value

    move, value = None, -math.inf
    for a in actions(board):
        a_, v_ = worst_move(drop(board, player, a, inplace=False), player, alpha, beta)
        if v_ > value:
            move, value = a, v_
            alpha = max(alpha, value)
        if value >= beta: return move, value
    return move, value

def worst_move(board, player, alpha=-math.inf, beta=+math.inf):
    """Returns worst move of player and its value"""
    value = utility(board, player)
    if value is not None: return None, value

    move, value = None, +math.inf
    for a in actions(board):
        a_, v_ = best_move(drop(board, other(player), a, inplace=False), player, alpha, beta)
        if v_ < value:
            move, value = a, v_
            beta = min(beta, value)
        if value <= alpha: return move, value
    return move, value

Experiment with some manually created boards (at least 5) to check if the agent spots winning opportunities.

In [9]:
boards = []

boards.append(np.array([
    [' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' '],
    ['o', 'o', 'o', ' '],
    ['x', 'x', 'x', ' ']
]))

boards.append(np.array([
    [' ', ' ', ' ', ' '],
    ['x', 'o', ' ', ' '],
    ['x', 'o', ' ', ' '],
    ['x', 'o', ' ', ' ']
]))

boards.append(np.array([
    [' ', ' ', ' ', ' '],
    [' ', ' ', 'x', 'o'],
    [' ', 'x', 'o', 'x'],
    ['x', 'o', 'o', 'o']
]))

boards.append(np.array([
    ['o', ' ', ' ', ' '],
    ['x', 'o', 'x', 'o'],
    ['o', 'o', 'x', 'x'],
    ['o', 'x', 'x', 'o']
]))

boards.append(np.array([
    ['x', ' ', ' ', ' '],
    ['o', 'x', ' ', 'o'],
    ['o', 'o', ' ', 'o'],
    ['x', 'x', ' ', 'x']
]))

for board in boards:
    print(best_move(board, 'x'))

# The minimax search appears to find a guaranteed win in all of these boards,
# but does not always choose the move which wins immediately.

(3, 1)
(0, 1)
(3, 1)
(2, 1)
(1, 1)


How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [8]:
%time minimax_agent(empty_board(shape=(4,4)), 'x')
%time minimax_agent(empty_board(shape=(4,5)), 'x')
%time minimax_agent(empty_board(shape=(4,6)), 'x')

Wall time: 656 ms
Wall time: 21.4 s
Wall time: 20min 49s


### Move ordering

Describe and implement a simple move ordering strategy. How does this strategy influence the time it takes to 
make a move?

In [10]:
# The number of 4-length lines which board spaces can be a part of 
# (excluding vertical lines) is described by the image linked below.
# https://markusthill.github.io/images/2018-02-09-connect-4-introduction-and-tree-search-algorithms/possibleChains.png

# Because of the board symmetry, only one quarter of it needs to be stored
line_counts = {
    (0,0):4, (0,1):6, (0,2):7, (0,3):8,
    (1,0):2, (1,1):4, (1,2):6, (1,3):8,
    (2,0):0, (2,1):2, (2,2):5, (2,3):8
}
def line_count(row, col):
    return line_counts[(-abs(2*row - 5) + 5)//2, abs(col - 3)]

# The new actions function sorts actions in order of which
# available spaces participate in the most lines.
def actions(board):
    # If zero or one moves have been made, only return the center column
    if np.unique(board).size < 2 : return [board.shape[1] // 2]
    spaces = {}
    for col in range(board.shape[1]):
        for row in reversed(range(board.shape[0])):
            if board[row, col] == ' ':
                spaces[col] = row
                break
    moves = [*spaces]
    return sorted(moves, key=lambda c: line_count(spaces[c], c))

%time minimax_agent(empty_board(shape=(4,4)), 'x')
%time minimax_agent(empty_board(shape=(4,5)), 'x')
%time minimax_agent(empty_board(shape=(4,6)), 'x')

Wall time: 704 ms
Wall time: 16.9 s
Wall time: 9min 48s


### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [11]:
print(play(minimax_agent, random_agent, size=(4,4)))

# The Minimax Search agent wins about 70% of games against a random agent.

{'x': 641, 'o': 174, 'draw': 185}


## Task 4: Heuristic Alpha-Beta Tree Search [3 points] 

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [12]:
from re import findall

@lru_cache
def heuristic_pattern(char, orientation, cols, length, space):
    o = orientation % 180
    if o == 0:    o = cols
    elif o == 45: o = 1
    elif o == 90: o = 0
    elif o == 135:o = -1
    pattern = '(?=' + (char + '.' * (cols - o)) * space + ' ' + ('.' * (cols - o) + char) * (length - space - 1) + ')'
    return pattern

@np_cache(maxsize=2**21)
def evaluate(board, player='x', length=4):
    """Heuristic for the utility of the board. Returns the score.

    If a player has already won, a scaled utility is returned.
    Otherwise, returns the number of lines that are one empty
    space away from winning the game for the player subtracted by
    similar lines for the opposing player."""
    u = utility(board, player)
    if u is not None:
        return u * board.size * 4, True

    # unravel board into string
    board_str = '|'.join(str(row.tostring().replace(b'\x00', b'').decode("utf-8")) for row in board)
    cols = board.shape[1]

    score = 0
    for i in range(length):
        for o in range(0, 180, 45):
            score += len(findall(heuristic_pattern(player, o, cols, length, i), board_str))
            score -= len(findall(heuristic_pattern(other(player), o, cols, length, i), board_str))
    return score, False

### Cutting off search 

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [13]:
import math

def heuristic_agent(board, player='x'):
    move, _ = good_move(board, player)
    drop(board, player, move)

def good_move(board, player, alpha=-math.inf, beta=+math.inf, depth=0, cutoff=10000000):
    """Returns heuristically best move of player and its value"""
    value, terminal = evaluate(board, player)
    if depth > cutoff or terminal:
        if terminal: alpha, beta = value, value
        return None, value

    move, value = None, -math.inf
    for a in actions(board):
        a_, v_ = bad_move(drop(board, player, a, inplace=False), player, alpha, beta, depth + 1, cutoff)
        if v_ > value:
            move, value = a, v_
            alpha = max(alpha, value)
        if value >= beta: return move, value
    return move, value

def bad_move(board, player, alpha=-math.inf, beta=+math.inf, depth=0, cutoff=10000000):
    """Returns heuristically best move of player and its value"""
    value, terminal = evaluate(board, player)
    if terminal:
        alpha, beta = value, value
        return None, value

    move, value = None, +math.inf
    for a in actions(board):
        a_, v_ = good_move(drop(board, player, a, inplace=False), player, alpha, beta, depth + 1, cutoff)
        if v_ < value:
            move, value = a, v_
            beta = min(alpha, value)
        if value <= beta: return move, value
    return move, value

Experiment with the same manually created boards as above to check if the agent spots wining opportunities.

In [14]:
for board in boards:
    print(good_move(board, 'x', cutoff=1))

print()

for board in boards:
    print(good_move(board, 'x', cutoff=10))

# With a cutoff of 1, the agent selects the moves that win immediately.
# However, with a greater cutoff, the agent selects the first move in
# the list of actions provided to it.

(3, 64)
(0, 64)
(3, 64)
(2, 64)
(2, 64)

(3, 64)
(3, 64)
(3, 64)
(3, 64)
(2, 64)


How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [15]:
%time heuristic_agent(empty_board(shape=(4,4)), 'x')
%time heuristic_agent(empty_board(shape=(4,5)), 'x')
%time heuristic_agent(empty_board(shape=(4,6)), 'x')

Wall time: 676 ms
Wall time: 208 ms
Wall time: 303 ms


### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [16]:
%time play(heuristic_agent, heuristic_agent, size=(6,7), n=1)

Wall time: 13.3 s


{'x': 1, 'o': 0, 'draw': 0}

## Challenge task [+ 1 bonus point]

Find another student and let your best agent play against the other student's best player. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [1 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

### Pure Monte Carlos Search

Implement Pure Monte Carlo Search and investigate how this search performs on the test boards that you have used above. 

In [17]:
def pure_monte_carlo_agent(board, player='x'):
    move = simulate(board, player)
    drop(board, player, move)

def playout(board, player, move):
    state = drop(board, player, move, inplace=False)
    current = other(player)
    u = utility(board, player)
    while u is None:
        state = drop(state, current, np.random.choice(actions(state)))
        current = other(current)
        u = utility(state, player)
    return u

def simulate(board, player='x', n=1000):
    """Returns the action with largest average utility as
    determined by a pure Monte Carlo search."""
    moves = actions(board)
    n = n // len(moves)
    averages = {}
    for a in moves:
        playouts = [playout(board, player, a) for i in range(n)]
        averages[a] = np.mean(playouts)
    print(averages)
    return max(averages, key=averages.get)

for board in boards:
    print(simulate(board, 'x'))
    print()

# The pure Monte Carlo search always finds the winning move immediately

3
0
3
2
2


### Best First Move

How would you determine what the best first move is? You can use Pure Monte Carlo Search or any algorithms 
that you have implemented above.

I would choose to determine the best first move by running a Pure Monte Carlo Search on an empty board for
as many simulations as I am willing to wait for. I would then determine the move with the highest average
utility to be the best move. Below, I use one million playthroughs of an empty board.

In [19]:
%time simulate(empty_board(), n=1000000)

Wall time: 40min 30s


3

In [ ]:
import C4Agent_AW
from importlib import reload
reload(C4Agent_AW)

play(C4Agent_AW.agent, C4Agent_AW.agent, n=1)